# PATSTAT Schema Explorer - TIP Environment

Dieses Notebook bietet eine umfassende Einführung in die PATSTAT Datenbank-Struktur und zeigt verschiedene Methoden zur Schema-Exploration.

## Inhalt
1. Grundlegende Verbindung und Setup
2. Schema-Übersicht mit SQLAlchemy Inspector
3. ORM-Modelle erkunden
4. Wichtige PATSTAT-Tabellen im Detail
5. Beziehungen zwischen Tabellen visualisieren
6. Praktische Abfrage-Beispiele
7. Hilfsfunktionen für die tägliche Arbeit

## 1. Setup und Import

In [ ]:
# Standard Imports
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# PATSTAT und SQLAlchemy
from patstat import Session, Base, engine
from sqlalchemy import inspect, MetaData, Table, text
from sqlalchemy.orm import Query

# Visualisierung
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Display Optionen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)

print("✅ Setup erfolgreich abgeschlossen")
print(f"📅 Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Datenbank-Verbindung testen

In [ ]:
# Session erstellen und Verbindung testen
session = Session()

try:
    # Einfache Test-Abfrage
    result = session.execute(text("SELECT 1 as test")).fetchone()
    print("✅ Datenbankverbindung erfolgreich!")
    
    # PATSTAT Version prüfen (über eine Beispiel-Abfrage)
    count_result = session.execute(
        text("SELECT COUNT(*) FROM tls201_appln WHERE appln_id < 100")
    ).fetchone()
    print(f"✅ PATSTAT Zugriff funktioniert (Test-Count: {count_result[0]})")
    
except Exception as e:
    print(f"❌ Fehler bei der Verbindung: {e}")
finally:
    session.close()

## 3. Schema-Übersicht mit SQLAlchemy Inspector

In [ ]:
# Inspector erstellen
inspector = inspect(engine)

# Alle Tabellen abrufen
all_tables = inspector.get_table_names()
print(f"📊 Anzahl gefundener Tabellen: {len(all_tables)}\n")

# Tabellen nach Kategorie gruppieren
tls_tables = {}
for table in all_tables:
    if table.startswith('tls'):
        prefix = table[:6]  # z.B. 'tls201'
        category = ''
        if prefix.startswith('tls2'):
            category = 'Data Tables (2xx)'
        elif prefix.startswith('tls8'):
            category = 'EPO Reference Tables (8xx)'
        elif prefix.startswith('tls9'):
            category = 'External Reference Tables (9xx)'
        else:
            category = 'Other'
        
        if category not in tls_tables:
            tls_tables[category] = []
        tls_tables[category].append(table)

# Kategorisierte Ausgabe
for category, tables in sorted(tls_tables.items()):
    print(f"\n{'='*50}")
    print(f"📁 {category}: {len(tables)} Tabellen")
    print(f"{'='*50}")
    for i, table in enumerate(sorted(tables)[:10], 1):  # Erste 10 anzeigen
        print(f"  {i:2}. {table}")
    if len(tables) > 10:
        print(f"  ... und {len(tables)-10} weitere")

## 4. Die wichtigsten PATSTAT-Tabellen im Detail

In [ ]:
# Definiere die Kern-Tabellen von PATSTAT
core_tables = {
    'Applications': {
        'tls201_appln': 'Haupttabelle für Patentanmeldungen',
        'tls202_appln_title': 'Titel der Anmeldungen',
        'tls203_appln_abstr': 'Abstracts der Anmeldungen'
    },
    'Publications': {
        'tls211_pat_publn': 'Publikationen zu Anmeldungen'
    },
    'Persons': {
        'tls206_person': 'Bereinigte Personendaten',
        'tls207_pers_appln': 'Verknüpfung Person-Anmeldung',
        'tls226_person_orig': 'Original Personendaten',
        'tls227_pers_publn': 'Verknüpfung Person-Publikation'
    },
    'Classifications': {
        'tls209_appln_ipc': 'IPC Klassifikationen',
        'tls224_appln_cpc': 'CPC Klassifikationen',
        'tls230_appln_techn_field': 'Technologiefelder'
    },
    'Citations': {
        'tls212_citation': 'Zitierungen',
        'tls215_citn_categ': 'Zitierungs-Kategorien',
        'tls214_npl_publn': 'Non-Patent Literature'
    },
    'Relations': {
        'tls204_appln_prior': 'Prioritäten',
        'tls216_appln_contn': 'Fortsetzungen/Teilungen',
        'tls205_tech_rel': 'Technische Beziehungen'
    },
    'Legal Events': {
        'tls231_inpadoc_legal_event': 'Rechtliche Ereignisse',
        'tls803_legal_event_code': 'Event Code Definitionen'
    }
}

# Erstelle DataFrame für Übersicht
table_overview = []
for category, tables in core_tables.items():
    for table_name, description in tables.items():
        if table_name in all_tables:
            table_overview.append({
                'Kategorie': category,
                'Tabelle': table_name,
                'Beschreibung': description,
                'Vorhanden': '✅'
            })
        else:
            table_overview.append({
                'Kategorie': category,
                'Tabelle': table_name,
                'Beschreibung': description,
                'Vorhanden': '❌'
            })

df_overview = pd.DataFrame(table_overview)
print("\n📋 PATSTAT Kern-Tabellen Übersicht:")
print("="*80)
display(df_overview)

## 5. Schema-Details für ausgewählte Tabellen

In [ ]:
def analyze_table_schema(table_name, inspector, limit=15):
    """Analysiert das Schema einer Tabelle im Detail"""
    
    if table_name not in inspector.get_table_names():
        print(f"❌ Tabelle '{table_name}' nicht gefunden")
        return None
    
    print(f"\n{'='*80}")
    print(f"📊 Schema-Analyse für: {table_name}")
    print(f"{'='*80}")
    
    # Spalten-Informationen
    columns = inspector.get_columns(table_name)
    print(f"\n📝 Spalten ({len(columns)} insgesamt, erste {min(limit, len(columns))} angezeigt):")
    print("-"*60)
    
    col_data = []
    for i, col in enumerate(columns[:limit], 1):
        col_data.append({
            'Nr': i,
            'Spaltenname': col['name'],
            'Datentyp': str(col['type']),
            'Nullable': '✓' if col['nullable'] else '✗',
            'Default': col.get('default', '-')
        })
    
    df_cols = pd.DataFrame(col_data)
    display(df_cols)
    
    # Primary Keys
    pk = inspector.get_pk_constraint(table_name)
    if pk and pk['constrained_columns']:
        print(f"\n🔑 Primary Key(s): {', '.join(pk['constrained_columns'])}")
    
    # Foreign Keys
    fks = inspector.get_foreign_keys(table_name)
    if fks:
        print(f"\n🔗 Foreign Keys ({len(fks)}):")
        for fk in fks[:5]:  # Erste 5 FKs anzeigen
            print(f"   • {', '.join(fk['constrained_columns'])} → "
                  f"{fk['referred_table']}({', '.join(fk['referred_columns'])})")
    
    # Indexes
    indexes = inspector.get_indexes(table_name)
    if indexes:
        print(f"\n📍 Indexes ({len(indexes)}):")
        for idx in indexes[:3]:  # Erste 3 Indexes
            print(f"   • {idx['name']}: {', '.join(idx['column_names'])} "
                  f"(unique: {idx['unique']})")
    
    return columns

# Analysiere die wichtigste Tabelle: tls201_appln
columns_appln = analyze_table_schema('tls201_appln', inspector)

In [ ]:
# Analysiere weitere wichtige Tabellen
important_tables = ['tls211_pat_publn', 'tls206_person', 'tls209_appln_ipc']

for table in important_tables:
    analyze_table_schema(table, inspector, limit=10)

## 6. Beziehungen zwischen Tabellen visualisieren

In [ ]:
# Sammle alle Foreign Key Beziehungen
def get_table_relationships(inspector, focus_tables=None):
    """Extrahiert alle FK-Beziehungen zwischen Tabellen"""
    relationships = []
    
    tables_to_check = focus_tables or inspector.get_table_names()
    
    for table in tables_to_check:
        if not table.startswith('tls'):
            continue
            
        fks = inspector.get_foreign_keys(table)
        for fk in fks:
            relationships.append({
                'from_table': table,
                'from_columns': ', '.join(fk['constrained_columns']),
                'to_table': fk['referred_table'],
                'to_columns': ', '.join(fk['referred_columns'])
            })
    
    return pd.DataFrame(relationships)

# Fokus auf Application-bezogene Tabellen
app_related_tables = [
    'tls201_appln', 'tls202_appln_title', 'tls203_appln_abstr',
    'tls211_pat_publn', 'tls207_pers_appln', 'tls209_appln_ipc',
    'tls212_citation', 'tls204_appln_prior'
]

df_relationships = get_table_relationships(inspector, app_related_tables)

print("\n🔗 Tabellen-Beziehungen (Foreign Keys):")
print("="*80)
display(df_relationships.head(15))

In [ ]:
# Visualisiere die Beziehungen als Netzwerk-Diagramm (vereinfacht)
import networkx as nx

def visualize_relationships(df_rel):
    """Erstellt ein einfaches Netzwerk-Diagramm der Tabellenbeziehungen"""
    
    # Erstelle gerichteten Graphen
    G = nx.DiGraph()
    
    # Füge Kanten hinzu
    for _, row in df_rel.iterrows():
        G.add_edge(row['from_table'], row['to_table'])
    
    # Visualisierung
    plt.figure(figsize=(14, 10))
    
    # Layout berechnen
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Zeichne Knoten und Kanten
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightblue', alpha=0.7)
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, 
                          arrowsize=20, alpha=0.5, arrowstyle='->')
    
    plt.title('PATSTAT Tabellen-Beziehungen (Foreign Keys)', fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Statistiken
    print(f"\n📊 Netzwerk-Statistiken:")
    print(f"  • Anzahl Tabellen (Knoten): {G.number_of_nodes()}")
    print(f"  • Anzahl Beziehungen (Kanten): {G.number_of_edges()}")
    
    # Zentrale Tabellen identifizieren
    in_degree = dict(G.in_degree())
    out_degree = dict(G.out_degree())
    
    print(f"\n🎯 Zentrale Tabellen (viele eingehende Referenzen):")
    for table, degree in sorted(in_degree.items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"  • {table}: {degree} eingehende Referenzen")
    
    print(f"\n🔄 Tabellen mit vielen Referenzen (ausgehend):")
    for table, degree in sorted(out_degree.items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"  • {table}: {degree} ausgehende Referenzen")

# Visualisiere das Beziehungsnetzwerk
if not df_relationships.empty:
    visualize_relationships(df_relationships)

## 7. Praktische Abfrage-Beispiele

In [ ]:
# Beispiel 1: Basis-Statistiken für eine Tabelle
def get_table_statistics(table_name, session):
    """Holt grundlegende Statistiken für eine Tabelle"""
    
    print(f"\n📊 Statistiken für {table_name}:")
    print("="*50)
    
    try:
        # Anzahl Zeilen (mit LIMIT für Performance)
        count_query = f"SELECT COUNT(*) FROM {table_name} WHERE ROWNUM <= 1000000"
        result = session.execute(text(count_query)).fetchone()
        print(f"  • Zeilen (max 1M gezählt): {result[0]:,}")
        
        # Beispieldaten
        sample_query = f"SELECT * FROM {table_name} WHERE ROWNUM <= 5"
        df_sample = pd.read_sql(sample_query, session.bind)
        
        print(f"  • Spalten: {len(df_sample.columns)}")
        print(f"  • Beispiel-Daten:")
        display(df_sample)
        
        return df_sample
        
    except Exception as e:
        print(f"  ❌ Fehler: {e}")
        return None

# Test mit tls201_appln
session = Session()
df_sample = get_table_statistics('tls201_appln', session)
session.close()

In [ ]:
# Beispiel 2: Verknüpfte Abfrage über mehrere Tabellen
def get_application_details(appln_id, session):
    """Holt Details zu einer spezifischen Anmeldung"""
    
    query = f"""
    SELECT 
        a.appln_id,
        a.appln_auth,
        a.appln_nr,
        a.appln_filing_date,
        a.appln_kind,
        a.granted,
        t.appln_title,
        a.docdb_family_id,
        a.inpadoc_family_id
    FROM 
        tls201_appln a
        LEFT JOIN tls202_appln_title t ON a.appln_id = t.appln_id
    WHERE 
        a.appln_id = :appln_id
    """
    
    result = pd.read_sql(query, session.bind, params={'appln_id': appln_id})
    return result

# Test mit einer Beispiel-ID
session = Session()
test_appln_id = 12345  # Beispiel-ID
df_details = get_application_details(test_appln_id, session)

if not df_details.empty:
    print(f"\n📋 Details für Application ID {test_appln_id}:")
    display(df_details.T)  # Transponiert für bessere Lesbarkeit
else:
    print(f"Keine Daten für ID {test_appln_id} gefunden")
    
session.close()

## 8. Hilfsfunktionen für die Schema-Arbeit

In [ ]:
class PatstatSchemaHelper:
    """Hilfsklasse für PATSTAT Schema-Operationen"""
    
    def __init__(self, engine):
        self.engine = engine
        self.inspector = inspect(engine)
        self.metadata = MetaData()
    
    def search_tables(self, pattern):
        """Sucht Tabellen nach Namens-Pattern"""
        tables = self.inspector.get_table_names()
        matching = [t for t in tables if pattern.lower() in t.lower()]
        return matching
    
    def search_columns(self, column_pattern, table_pattern=None):
        """Sucht Spalten nach Namen über alle Tabellen"""
        results = []
        
        tables = self.inspector.get_table_names()
        if table_pattern:
            tables = [t for t in tables if table_pattern.lower() in t.lower()]
        
        for table in tables:
            columns = self.inspector.get_columns(table)
            for col in columns:
                if column_pattern.lower() in col['name'].lower():
                    results.append({
                        'table': table,
                        'column': col['name'],
                        'type': str(col['type']),
                        'nullable': col['nullable']
                    })
        
        return pd.DataFrame(results)
    
    def get_table_size(self, table_name, session):
        """Schätzt die Größe einer Tabelle"""
        try:
            # Für Oracle-basierte Systeme
            query = f"""
            SELECT 
                num_rows as row_count,
                blocks * 8 / 1024 as size_mb
            FROM 
                user_tables 
            WHERE 
                UPPER(table_name) = UPPER(:table_name)
            """
            result = session.execute(text(query), {'table_name': table_name}).fetchone()
            if result:
                return {'rows': result[0], 'size_mb': result[1]}
        except:
            pass
        
        # Fallback: COUNT
        try:
            count = session.execute(
                text(f"SELECT COUNT(*) FROM {table_name} WHERE ROWNUM <= 10000")
            ).scalar()
            return {'rows': count, 'size_mb': None}
        except:
            return {'rows': None, 'size_mb': None}
    
    def export_schema_to_excel(self, filename='patstat_schema.xlsx'):
        """Exportiert das komplette Schema nach Excel"""
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            # Übersicht aller Tabellen
            tables = self.inspector.get_table_names()
            df_tables = pd.DataFrame({'table_name': tables})
            df_tables.to_excel(writer, sheet_name='Tables', index=False)
            
            # Details für wichtige Tabellen
            important_tables = ['tls201_appln', 'tls211_pat_publn', 'tls206_person']
            for table in important_tables:
                if table in tables:
                    columns = self.inspector.get_columns(table)
                    df_cols = pd.DataFrame(columns)
                    # Sheet-Name kürzen auf max 31 Zeichen (Excel-Limit)
                    sheet_name = table[:31]
                    df_cols.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"✅ Schema exportiert nach: {filename}")

# Instanz erstellen und testen
helper = PatstatSchemaHelper(engine)

# Beispiel 1: Suche nach Tabellen mit 'person' im Namen
print("\n🔍 Tabellen mit 'person' im Namen:")
person_tables = helper.search_tables('person')
for table in person_tables:
    print(f"  • {table}")

# Beispiel 2: Suche nach Spalten mit 'family' im Namen
print("\n🔍 Spalten mit 'family' im Namen (erste 10):")
df_family_cols = helper.search_columns('family', 'tls')
display(df_family_cols.head(10))

## 9. Cheat Sheet: Wichtigste PATSTAT Tabellen & Spalten

In [ ]:
# PATSTAT Cheat Sheet als strukturiertes Dictionary
patstat_cheatsheet = {
    'tls201_appln': {
        'Beschreibung': 'Haupttabelle für Patentanmeldungen',
        'Primary Key': 'appln_id',
        'Wichtige Spalten': [
            'appln_id: Eindeutige ID der Anmeldung',
            'appln_auth: Anmeldeamt (z.B. EP, US, DE)',
            'appln_nr: Anmeldenummer',
            'appln_filing_date: Anmeldedatum',
            'appln_kind: Art der Anmeldung',
            'granted: Patent erteilt (Y/N)',
            'docdb_family_id: DOCDB Familie',
            'inpadoc_family_id: INPADOC Familie',
            'earliest_filing_date: Frühestes Anmeldedatum'
        ]
    },
    'tls211_pat_publn': {
        'Beschreibung': 'Publikationen zu Anmeldungen',
        'Primary Key': 'pat_publn_id',
        'Wichtige Spalten': [
            'pat_publn_id: Eindeutige Publikations-ID',
            'appln_id: Verknüpfung zur Anmeldung',
            'publn_auth: Publikationsamt',
            'publn_nr: Publikationsnummer',
            'publn_kind: Art der Publikation (A1, B1, etc.)',
            'publn_date: Publikationsdatum',
            'publn_first_grant: Erste Erteilung (Y/N)'
        ]
    },
    'tls206_person': {
        'Beschreibung': 'Bereinigte Personendaten (Anmelder, Erfinder)',
        'Primary Key': 'person_id',
        'Wichtige Spalten': [
            'person_id: Eindeutige Personen-ID',
            'person_name: Name der Person/Organisation',
            'person_ctry_code: Ländercode',
            'psn_name: Standardisierter Name',
            'psn_sector: Sektor (Individual, Company, etc.)'
        ]
    },
    'tls207_pers_appln': {
        'Beschreibung': 'Verknüpfung Person-Anmeldung',
        'Primary Key': 'person_id, appln_id, applt_seq_nr, invt_seq_nr',
        'Wichtige Spalten': [
            'person_id: Verknüpfung zu tls206_person',
            'appln_id: Verknüpfung zu tls201_appln',
            'applt_seq_nr: Sequenz-Nr. als Anmelder (>0 = ist Anmelder)',
            'invt_seq_nr: Sequenz-Nr. als Erfinder (>0 = ist Erfinder)'
        ]
    },
    'tls209_appln_ipc': {
        'Beschreibung': 'IPC-Klassifikationen',
        'Primary Key': 'appln_id, ipc_class_symbol',
        'Wichtige Spalten': [
            'appln_id: Verknüpfung zur Anmeldung',
            'ipc_class_symbol: IPC-Symbol (z.B. H01L 21/00)',
            'ipc_position: Position (F=First, L=Later)',
            'ipc_value: Wert (I=Invention, N=Non-invention)'
        ]
    },
    'tls212_citation': {
        'Beschreibung': 'Patent-Zitierungen',
        'Primary Key': 'citn_id',
        'Wichtige Spalten': [
            'pat_publn_id: Zitierende Publikation',
            'cited_pat_publn_id: Zitierte Patent-Publikation',
            'cited_appln_id: Zitierte Anmeldung',
            'citn_origin: Ursprung der Zitierung'
        ]
    }
}

# Formatierte Ausgabe als Referenz
print("📚 PATSTAT CHEAT SHEET")
print("=" * 80)

for table_name, info in patstat_cheatsheet.items():
    print(f"\n🗂️  {table_name}")
    print(f"   {info['Beschreibung']}")
    print(f"   🔑 Primary Key: {info['Primary Key']}")
    print(f"   📋 Wichtige Spalten:")
    for col in info['Wichtige Spalten'][:5]:  # Erste 5 anzeigen
        print(f"      • {col}")

## 10. Erweiterte Analyse-Funktionen

In [ ]:
def create_query_builder(base_table):
    """Erstellt einen einfachen Query Builder für PATSTAT"""
    
    class QueryBuilder:
        def __init__(self, table):
            self.base_table = table
            self.select_cols = ['*']
            self.joins = []
            self.where_conditions = []
            self.limit_value = None
        
        def select(self, *columns):
            self.select_cols = list(columns)
            return self
        
        def join(self, table, on_condition):
            self.joins.append(f"LEFT JOIN {table} ON {on_condition}")
            return self
        
        def where(self, condition):
            self.where_conditions.append(condition)
            return self
        
        def limit(self, n):
            self.limit_value = n
            return self
        
        def build(self):
            query = f"SELECT {', '.join(self.select_cols)} FROM {self.base_table}"
            
            for join in self.joins:
                query += f" {join}"
            
            if self.where_conditions:
                query += " WHERE " + " AND ".join(self.where_conditions)
            
            if self.limit_value:
                query += f" AND ROWNUM <= {self.limit_value}"
            
            return query
    
    return QueryBuilder(base_table)

# Beispiel: Query Builder verwenden
query = (create_query_builder('tls201_appln')
         .select('a.appln_id', 'a.appln_auth', 'a.appln_filing_date', 't.appln_title')
         .join('tls202_appln_title t', 'a.appln_id = t.appln_id')
         .where("a.appln_auth = 'EP'")
         .where("a.appln_filing_date >= DATE '2020-01-01'")
         .limit(10)
         .build())

print("🔨 Generierte Query:")
print(query)

In [ ]:
# Funktion zum Generieren von Basis-Queries für häufige Analysen
def generate_analysis_queries():
    """Generiert Standard-Analyse-Queries für PATSTAT"""
    
    queries = {
        'recent_patents': """
            -- Neueste Patentanmeldungen
            SELECT 
                a.appln_id,
                a.appln_auth,
                a.appln_nr,
                a.appln_filing_date,
                t.appln_title
            FROM 
                tls201_appln a
                LEFT JOIN tls202_appln_title t ON a.appln_id = t.appln_id
            WHERE 
                a.appln_filing_date >= DATE '2023-01-01'
                AND ROWNUM <= 100
            ORDER BY 
                a.appln_filing_date DESC
        """,
        
        'top_applicants': """
            -- Top Anmelder nach Anzahl Anmeldungen
            SELECT 
                p.person_name,
                p.person_ctry_code,
                COUNT(DISTINCT pa.appln_id) as application_count
            FROM 
                tls207_pers_appln pa
                JOIN tls206_person p ON pa.person_id = p.person_id
            WHERE 
                pa.applt_seq_nr > 0  -- Ist Anmelder
                AND ROWNUM <= 10000
            GROUP BY 
                p.person_name, p.person_ctry_code
            ORDER BY 
                application_count DESC
        """,
        
        'technology_distribution': """
            -- Verteilung nach IPC-Klassen
            SELECT 
                SUBSTR(ipc_class_symbol, 1, 4) as ipc_section,
                COUNT(DISTINCT appln_id) as application_count
            FROM 
                tls209_appln_ipc
            WHERE 
                ipc_position = 'F'  -- First/Main classification
                AND ROWNUM <= 10000
            GROUP BY 
                SUBSTR(ipc_class_symbol, 1, 4)
            ORDER BY 
                application_count DESC
        """,
        
        'citation_network': """
            -- Zitierungs-Netzwerk
            SELECT 
                c.pat_publn_id as citing_publication,
                c.cited_pat_publn_id as cited_publication,
                p1.publn_date as citing_date,
                p2.publn_date as cited_date
            FROM 
                tls212_citation c
                JOIN tls211_pat_publn p1 ON c.pat_publn_id = p1.pat_publn_id
                LEFT JOIN tls211_pat_publn p2 ON c.cited_pat_publn_id = p2.pat_publn_id
            WHERE 
                c.cited_pat_publn_id > 0
                AND p1.publn_date >= DATE '2023-01-01'
                AND ROWNUM <= 100
        """
    }
    
    return queries

# Queries anzeigen
analysis_queries = generate_analysis_queries()

print("📊 Verfügbare Analyse-Queries:")
print("=" * 50)
for name, query in analysis_queries.items():
    print(f"\n📌 {name}:")
    print(f"   Erste 200 Zeichen der Query:")
    print(f"   {query[:200].strip()}...")

## 11. Zusammenfassung und nächste Schritte

In [ ]:
print("""
🎯 ZUSAMMENFASSUNG
================================================================================

Du hast jetzt eine umfassende Einführung in die PATSTAT-Datenbankstruktur erhalten:

✅ Gelernte Konzepte:
   • SQLAlchemy Inspector zur Schema-Exploration
   • PATSTAT Tabellen-Struktur und Namenskonventionen
   • Wichtigste Tabellen und deren Beziehungen
   • Foreign Key Beziehungen und Datenmodell
   • Praktische Abfrage-Beispiele

📚 Wichtigste PATSTAT-Tabellen:
   • tls201_appln: Patentanmeldungen (zentrale Tabelle)
   • tls211_pat_publn: Publikationen
   • tls206_person: Personen (Anmelder, Erfinder)
   • tls209_appln_ipc: IPC-Klassifikationen
   • tls212_citation: Zitierungen

🚀 Nächste Schritte:
   1. Erkunde spezifische Technologiefelder mit IPC/CPC-Codes
   2. Analysiere Patent-Familien (DOCDB, INPADOC)
   3. Erstelle Zitierungs-Netzwerke
   4. Untersuche geografische Verteilungen
   5. Analysiere zeitliche Trends

💡 Tipps für effizientes Arbeiten:
   • Nutze immer WHERE-Klauseln mit ROWNUM für große Tabellen
   • Verwende Indizes (appln_id, person_id, pat_publn_id)
   • Speichere häufig genutzte Queries als Views oder Funktionen
   • Dokumentiere deine Analysen gut

📖 Weitere Ressourcen:
   • EPO Data Catalog (im Projektwissen)
   • PATSTAT Online Manual
   • EPO Patent Information Services

================================================================================
""")

# Speichere wichtige Variablen für spätere Verwendung
print("\n💾 Wichtige Variablen in diesem Notebook:")
print("   • inspector: SQLAlchemy Inspector Objekt")
print("   • helper: PatstatSchemaHelper Instanz")
print("   • patstat_cheatsheet: Dictionary mit Tabellen-Infos")
print("   • analysis_queries: Vorgefertigte Analyse-Queries")
print("\n✨ Viel Erfolg bei deinen PATSTAT-Analysen!")